<a href="https://colab.research.google.com/github/5harad/API201-students/blob/main/lectures/lecture-5-code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lecture 5: Bayes' rule

In [31]:
library(tidyverse)

## Part 1: Simulating cancer testing 

In [32]:
set.seed(1)

In [33]:
rbernoulli(5, 0.5)

[1] FALSE FALSE  TRUE  TRUE FALSE

In [34]:
rbernoulli(5, 0.5)

[1]  TRUE  TRUE  TRUE  TRUE FALSE

In [35]:
rbernoulli(5, c(0.75, 0.1, 0.1, .75, 0.1))

[1] FALSE FALSE FALSE  TRUE FALSE

In [36]:
rbernoulli(5, c(0.75, 0.1, 0.1, .75, 0.1))

[1]  TRUE FALSE  TRUE  TRUE FALSE

In [37]:
set.seed(22)

In [38]:
num_patients <- 1e6
incidence <- 0.009
tpr <- 0.75
tnr <- 0.9

In [39]:
cancer <- rbernoulli(num_patients, incidence)
head(cancer)

[1] FALSE FALSE  TRUE FALSE FALSE FALSE

In [40]:
p_pos <- if_else(cancer, tpr, 1-tnr)
head(p_pos)

[1] 0.10 0.10 0.75 0.10 0.10 0.10

In [41]:
set.seed(22)

In [42]:
patients <- tibble(
    cancer = rbernoulli(num_patients, incidence),
    test = rbernoulli(num_patients, if_else(cancer, tpr, 1-tnr))
)

In [43]:
head(patients)

cancer,test
<lgl>,<lgl>
FALSE,FALSE
FALSE,TRUE
TRUE,TRUE
FALSE,FALSE
FALSE,FALSE
FALSE,FALSE


In [45]:
patients %>%
    filter(test == TRUE) %>%
    summarize(p = mean(cancer))

p
<dbl>
0.06453958
